In [1]:
import os
import nltk
import re
import numpy as np
import copy
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim
import torch.utils.tensorboard
from torch.utils.tensorboard import SummaryWriter
from torch.autograd import Variable
device = 'cuda' if torch.cuda.is_available() else 'cpu'



In [2]:
nltk.download('punkt')

documents = {}
labels = {}
p = re.compile(r"(\b[-']\b)|[\W_]")

# open main data
for file in os.listdir("/home/valeriya/Desktop/UMD/Computational_linguistic/Project/Hulth2003/Hulth2003/Training"):
    if file.endswith(".abstr"):
        content = open(("%s/%s" % ('/home/valeriya/Desktop/UMD/Computational_linguistic/Project/Hulth2003/Hulth2003/Training', file)), "r").read()
        tmp=[]
        for s in content.split('.'):
            tmp.append( p.sub(lambda m: (m.group(1) if m.group(1) else " "), s) )
        documents[file.split('.')[0]] = tmp
        
# open labels        
for file in os.listdir("/home/valeriya/Desktop/UMD/Computational_linguistic/Project/Hulth2003/Hulth2003/Training"):
    if file.endswith(".uncontr"):
        content = open(("%s/%s" % ('/home/valeriya/Desktop/UMD/Computational_linguistic/Project/Hulth2003/Hulth2003/Training', file)), "r").read()
        tmp=[]
        for s in content.split('.'):
            tmp.append( p.sub(lambda m: (m.group(1) if m.group(1) else " "), s) )
        labels[file.split('.')[0]] = tmp  

[nltk_data] Downloading package punkt to /home/valeriya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# tokenize document
tokenized_documents = {}
for num, ctt in documents.items():
    tokenized_documents[num] = []
    for sentence in ctt:
        words = nltk.word_tokenize(sentence.lower())
        tokenized_documents[num].append(words)

# tokenize labels
tokenized_labels = {}
for num, ctt in labels.items():
    tokenized_labels[num] = []
    for sentence in ctt:
        words = nltk.word_tokenize(sentence.lower())
        tokenized_labels[num].append(words)   

In [4]:
# Here we preprocess labels: we label each word in each sentence with 2 if it is not a key-phrase word, 
# with 0 if it is a first word in key-phrase and 1 if it is not a first word in key-phrase.  

# function finds index of element in list of lists
def index(lab, target):
    for i,phr in enumerate(lab):
        for j, w in enumerate(phr):
            if w == target:
                return (j)
    return (None, None)

# create dictionary of labels associated to words
class_labels = copy.deepcopy(tokenized_documents)
for document in tokenized_documents:
    text = tokenized_documents[document]
    lab = tokenized_labels[document]
    lab_flattened = [val for sublist in lab for val in sublist]
    for i, sentence in enumerate(text): 
        for j, word in enumerate(sentence): 
            is_keyphrase = word in lab_flattened
            if is_keyphrase:
                if index(tokenized_labels[document], word) == 0:
                    class_labels[document][i][j] = 0
                else:
                    class_labels[document][i][j] = 1
            else:
                class_labels[document][i][j] = 2

# GLOVE embeddings

In [5]:
# here we download pretrained glove embeddings
import numpy as np
embeddings = dict()
embed_size = 100
f = open('glove.6B/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings[word] = coefs
f.close()

glove_size = 100

In [22]:
# our data
X = [sent for doc in copy.deepcopy(tokenized_documents).values() for sent in doc if sent!= []]
y = [sent for doc in copy.deepcopy(class_labels).values() for sent in doc if sent!=[]]

# our vocab: all the words in all abstracts
target_vocab = list(set([item for sublist in X for item in sublist]))
# dictionary with all words and their indices
vocab_ind_dict = dict(zip(target_vocab, range(0, len(target_vocab)))) 


# create matrix of glove vectors + random vectors for the words which are in vocab but not in glove

matrix_len = len(target_vocab)
weights_glove = np.zeros((matrix_len, glove_size))

for i, word in enumerate(target_vocab):
    try: 
        weights_glove[i] = embeddings[word]
    except KeyError:
        weights_glove[i] = np.random.normal(scale=0.6, size=(glove_size, ))

# replace words in our data with their indices
for i, sent in enumerate(X):
    for j, word in enumerate(sent):
        X[i][j] = vocab_ind_dict[word]

# BiLSTM Net

In [64]:
class LSTM(nn.Module):
    def __init__(self, nb_layers = 1, nb_lstm_units=150, nb_lin_units=150, embedding_dim=100, batch_size=1):
        super(LSTM, self).__init__()
        
        self.vocab = target_vocab
        self.tags = {'first': 0, 'inside': 1, 'no': 2}
        self.nb_lstm_layers = nb_layers
        self.nb_lstm_units = nb_lstm_units
        self.nb_lin_units = nb_lin_units
        self.embedding_dim = embedding_dim
        self.batch_size = batch_size
        self.nb_tags = len(self.tags)
        nb_vocab_words = len(self.vocab)

        
        # Embedding layer
        self.word_embedding = nn.Embedding(
            num_embeddings=nb_vocab_words,
            embedding_dim=self.embedding_dim)
        self.word_embedding.load_state_dict({'weight': torch.Tensor(weights_glove)})
        self.word_embedding.weight.requires_grad = False

        # LSTM
        self.lstm = nn.LSTM(
            input_size=self.embedding_dim,
            hidden_size=self.nb_lstm_units,
            num_layers=self.nb_lstm_layers,
            batch_first=True, bidirectional=True)
        
        # dropout layer
        self.drop = torch.nn.Dropout(p=0.25)

        # linear layers
        self.lin1 = nn.Linear(2*self.nb_lstm_units, self.nb_lin_units)
        self.lin2 = nn.Linear(self.nb_lstm_units, self.nb_tags)

    def init_hidden(self):
        # the weights are of the form (nb_layers, batch_size, nb_lstm_units)
        hidden_a = torch.randn(2, self.batch_size, self.nb_lstm_units).to(device)
        hidden_b = torch.randn(2, self.batch_size, self.nb_lstm_units).to(device)
        hidden_a = Variable(hidden_a)
        hidden_b = Variable(hidden_b)

        return (hidden_a, hidden_b)

    def forward(self, X):
        
        self.hidden = self.init_hidden()
        X = self.word_embedding(X)
        batch_size, seq_len, _ = X.size()

        X, self.hidden = self.lstm(X, self.hidden)
        X = self.drop(X)
        X = X.contiguous()
        X = X.view(-1, X.shape[2])

        X = F.relu(self.lin1(X))
        X = F.relu(self.lin2(X))

        X = X.view(batch_size, self.nb_tags, -1)

        return X

In [65]:
net = LSTM().to(device)
criterion= torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
tb = SummaryWriter()

In [66]:
# training function should be here 
# Something like 

def train(epoch):
    train_loss = 0.0
    total = 0
    correct = 0
    for i, sent in enumerate(X):
        inputs = torch.LongTensor(sent).view([1, len(sent)]).to(device)
        labels = torch.LongTensor(y[i]).view([1, len(y[i])]).to(device)     
        # zero the parameter gradients
        optimizer.zero_grad()
        # compute predictions
        outputs = (net(inputs))
        # compute loss
        loss = criterion(outputs, labels)
        # compute gradients
        loss.backward()
        # update parameters
        optimizer.step()
        # statistics to display
        train_loss += loss.item()
        _,predicted = outputs.max(1)
        total += labels[0].size(0)
        correct += predicted.eq(labels).sum().item()

    acc = 100.*correct/total
    print('Train accuracy:', acc)
    print('Loss:', train_loss)
    tb.add_scalar('Loss', train_loss, epoch)
    tb.add_scalar('Accuracy', acc, epoch)
    return(train_loss)


def main(num_epochs):
    for epoch in range(num_epochs):
        print('\nEpoch: %d' % epoch)
        train_loss = train(epoch)
    tb.close()
        

In [ ]:
main(20)


Epoch: 0
Train accuracy: 73.72410269115234
Loss: 6296.4189068220485

Epoch: 1
Train accuracy: 74.9326437707101
Loss: 6416.993394136429

Epoch: 2
